In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import random
import math
data_train=pd.read_csv(r"/content/drive/MyDrive/trainMnist.csv")
data_test=pd.read_csv(r"/content/drive/MyDrive/trainMnist.csv")

data_train=np.array(data_train)
data_test=np.array(data_test)

nb_exemple,nb_pixels=data_train.shape
nb_exemple_test,nb_pixels_test=data_test.shape

np.random.shuffle(data_train)
np.random.shuffle(data_test)

data_train=data_train.T
Y_train=data_train[0]
X_train=data_train[1:nb_pixels]
X_train=X_train/255.

data_test=data_test.T
Y_test=data_test[0]
X_test=data_test[1:nb_pixels_test]
X_test=X_test/255.


def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1,b1,W2,b2,X):
    Z1=W1.dot(X) + b1
    A1=ReLU(Z1)
    Z2=W2.dot(A1) + b2
    A2=softmax(Z2)
    return Z1,A1,Z2,A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y


def ReLU_deriv(Z):
    return Z > 0
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    #erruer
    dZ2 = A2 - one_hot_Y
    #calculer deriver d'erruer hidden layer 2
    dW2 = 1 / 1000 * dZ2.dot(A1.T)
    db2 = 1 / 1000 * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / 1000 * dZ1.dot(X.T)
    db1 = 1 / 1000 * np.sum(dZ1)
    return dW1, db1, dW2, db2

#methode gradient
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

#methode momentieum
def update_parametrs(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha,beta,dV1,dV2,dVB1,dVB2):
    dV1=beta*dV1+(1-beta)*dW1
    W1 = W1 - alpha * dV1

    dVB1=beta*dVB1+(1-beta)*db1
    b1 = b1 - alpha * dVB1

    dV2=beta*dV2+(1-beta)*dW2
    W2 = W2 - alpha * dV2

    dVB2 = beta * dVB2 + (1 - beta) * db2
    b2 = b2 - alpha * dVB2
    return W1, b1, W2, b2,dV1,dV2,dVB1,dVB2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def loss_function(Y_train,Y_pre):
    one_hot_Y = one_hot(Y_train)
    return 0.5 * np.mean((Y_pre - one_hot_Y) ** 2)

# ***`Adam`***

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c):
    alpha = 0.001
    beta1 = 0.9
    beta2 = 0.999
    eps = 1e-8
    #
    # v1=v1+dW1**2
    # v2=v2+dW2**2
    # vb1=vb1+db1**2
    # vb2=vb2+db2**2

    # vraiiiiiiiiiii

    # Update the weights and biases
    W1 -= alpha * m1c / (np.sqrt(v1c) + eps)
    W2 -= alpha * m2c / (np.sqrt(v2c) + eps)
    b1 -= alpha * mb1c / (np.sqrt(vb1c) + eps)
    b2 -= alpha * mb2c / (np.sqrt(vb2c) + eps)

    # W1 = W1 - alpha * m1c * 1 / np.sqrt(v1c + eps)
    # W2 = W2 - alpha * m2c * 1 / np.sqrt(v2c + eps)
    # b1 = b1 - alpha * mb1c * 1 / np.sqrt(vb1c + eps)
    # b2 = b2 - alpha * mb2c * 1 / np.sqrt(vb2c + eps)
    # vraiiiiiiiiiiiii
    m1c = m1 / (1 - beta1)
    m2c = m2 / (1 - beta1)
    mb1c = mb1 / (1 - beta1)
    mb2c = mb2 / (1 - beta1)
    # vraiiiiiiiiiiiii
    v1c = v1 / (1 - beta2)
    v2c = v2 / (1 - beta2)
    vb1c = vb1 / (1 - beta2)
    vb2c = vb2 / (1 - beta2)
    # vraiiiiiiii


    # m1 = beta1 * m1 + (1 - beta1) * dW1
    # m2 = beta1 * m2 + (1 - beta1) * dW2
    # mb1 = beta1 * mb1 + (1 - beta1) * db1
    # mb2 = beta1 * mb2 + (1 - beta1) * db2
    #
    # v1 = beta2 * v1 + (1 - beta2) * (dW1**2)
    # v2 = beta2 * v2 + (1 - beta2) * (dW2**2)
    # vb1 = beta2 * vb1 + (1 - beta2) * (db1**2)
    # vb2 = beta2 * vb2 + (1 - beta2) * (db2**2)
    m1 = beta1 * m1 + (1 - beta1) * dW1
    mb1 = beta1 * mb1 + (1 - beta1) * db1
    m2 = beta1 * m2 + (1 - beta1) * dW2
    mb2 = beta1* mb2 + (1 - beta1) * db2
    # vraiiiiiiii
    v1 = beta2 * v1 + (1 - beta2) * (dW1**2)
    vb1 = beta2 * vb1 + (1 - beta2) *( db1**2)
    v2 = beta2 * v2 + (1 - beta2) * (dW2**2)
    vb2 = beta2* vb2 + (1 - beta2) * (db2**2)





    # beta = 0.9
    # W1 = W1 - alpha * v1
    # b1 = b1 - alpha * vb1
    # W2 = W2 - alpha * v2
    # b2 = b2 - alpha * vb2
    # v1 = beta * v1 + (1 - beta) * dW1
    # vb1 = beta * vb1 + (1 - beta) * db1
    # v2 = beta * v2 + (1 - beta) * dW2
    # vb2 = beta * vb2 + (1 - beta) * db2

    return W1, b1, W2, b2, dW1, db1, dW2, db2, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    v1 = np.zeros(W1.shape)
    vb1 = np.zeros(b1.shape)
    v2 = np.zeros(W2.shape)
    vb2 = np.zeros(b2.shape)

    v1c = np.zeros(W1.shape)
    vb1c = np.zeros(b1.shape)
    v2c = np.zeros(W2.shape)
    vb2c = np.zeros(b2.shape)

    m1c = np.zeros(W1.shape)
    mb1c = np.zeros(b1.shape)
    m2c = np.zeros(W2.shape)
    mb2c = np.zeros(b2.shape)

    m1 = np.zeros(W1.shape)
    mb1 = np.zeros(b1.shape)
    m2 = np.zeros(W2.shape)
    mb2 = np.zeros(b2.shape)


    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2, dW1, db1, dW2, db2, v1, v2, vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,\
        mb1c,mb2c = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2,
                                  vb1,vb2,m1,m2,mb1,mb2,v1c,v2c,vb1c,vb2c,m1c,m2c,mb1c,mb2c)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 50)

Iteration:  0
0.14504761904761904
Iteration:  10
0.18564285714285714
Iteration:  20
0.28969047619047616
Iteration:  30
0.34376190476190477
Iteration:  40
0.40714285714285714


# ALgo adagrad

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2):
    eps=0.00000001

    v1=v1+dW1**2
    v2=v2+dW2**2
    vb1=vb1+db1**2
    vb2=vb2+db2**2

    W1 = W1 - alpha * dW1 *1/ np.sqrt(v1 + eps)
    W2 = W2 - alpha * dW2 *1/ np.sqrt(v2 + eps)
    b1 = b1 - alpha * db1 *1/ np.sqrt(vb1 + eps)
    b2 = b2 - alpha * db2 *1/np.sqrt(vb2 + eps)

    # beta = 0.9
    # W1 = W1 - alpha * v1
    # b1 = b1 - alpha * vb1
    # W2 = W2 - alpha * v2
    # b2 = b2 - alpha * vb2
    # v1 = beta * v1 + (1 - beta) * dW1
    # vb1 = beta * vb1 + (1 - beta) * db1
    # v2 = beta * v2 + (1 - beta) * dW2
    # vb2 = beta * vb2 + (1 - beta) * db2

    return W1, b1, W2, b2, v1, v2, vb1, vb2

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2, w11, w22, b11, b22 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11,
                                                           b22)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 100)

Iteration:  0
0.06278571428571429
Iteration:  10
0.6269047619047619
Iteration:  20
0.7666428571428572
Iteration:  30
0.8136904761904762
Iteration:  40
0.8375476190476191
Iteration:  50
0.8595238095238096
Iteration:  60
0.8718571428571429
Iteration:  70
0.8811190476190476
Iteration:  80
0.8872857142857142
Iteration:  90
0.8887142857142857


## RMS **PROP**

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2):
    eps=0.00000001
    beta = 0.99
    v1=beta*v1+(1-beta)*dW1**2
    v2=beta*v2+(1-beta)*dW2**2
    vb1=beta*vb1+(1-beta)*db1**2
    vb2=beta*vb2+(1-beta)*db2**2
    W1 = W1 - alpha * dW1 *1/ np.sqrt(v1 + eps)
    W2 = W2 - alpha * dW2 *1/ np.sqrt(v2 + eps)
    b1 = b1 - alpha * db1 *1/ np.sqrt(vb1 + eps)
    b2 = b2 - alpha * db2 *1/np.sqrt(vb2 + eps)


    # W1 = W1 - alpha * v1
    # b1 = b1 - alpha * vb1
    # W2 = W2 - alpha * v2
    # b2 = b2 - alpha * vb2
    # v1 = beta * v1 + (1 - beta) * dW1
    # vb1 = beta * vb1 + (1 - beta) * db1
    # v2 = beta * v2 + (1 - beta) * dW2
    # vb2 = beta * vb2 + (1 - beta) * db2

    return W1, b1, W2, b2, v1, v2, vb1, vb2

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2, w11, w22, b11, b22 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11,
                                                           b22)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
0.11228571428571428
Iteration:  10
0.18907142857142858
Iteration:  20
0.19226190476190477
Iteration:  30
0.19373809523809524
Iteration:  40
0.1951190476190476
Iteration:  50
0.1962142857142857
Iteration:  60
0.19702380952380952
Iteration:  70
0.19807142857142856
Iteration:  80
0.1987857142857143
Iteration:  90
0.19938095238095238
Iteration:  100
0.19966666666666666
Iteration:  110
0.1998095238095238
Iteration:  120
0.19992857142857143
Iteration:  130
0.20002380952380952
Iteration:  140
0.2001904761904762
Iteration:  150
0.20023809523809524
Iteration:  160
0.20045238095238097
Iteration:  170
0.2005952380952381
Iteration:  180
0.20076190476190475
Iteration:  190
0.20083333333333334
Iteration:  200
0.20102380952380952
Iteration:  210
0.2012857142857143
Iteration:  220
0.20135714285714285
Iteration:  230
0.20164285714285715
Iteration:  240
0.20183333333333334
Iteration:  250
0.20176190476190475
Iteration:  260
0.20226190476190475
Iteration:  270
0.20216666666666666
Iteration:

# **AdaDelta algorithme **

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, v1, v2, vb1, vb2,D1,D2,Db1,Db2):
    w1_old = W1.copy()
    w2_old = W2.copy()
    b1_old = b1.copy()
    b2_old = b2.copy()

    eps=0.00000001
    beta = 0.9




    #
    # v1 = beta * v1 + (1 - beta) *( dW1 ** 2)
    # v2 = beta * v2 + (1 - beta) * (dW2 ** 2)
    # vb1 = beta * vb1 + (1 - beta) * (db1 ** 2)
    # vb2 = beta * vb2 + (1 - beta) * (db2 ** 2)

    v1 = beta * v1 + (1 - beta) * (dW1 ** 2)
    v2 = beta * v2 + (1 - beta) * (dW2 ** 2)
    vb1 = beta * vb1 + (1 - beta) * (db1 ** 2)
    vb2 = beta * vb2 + (1 - beta) * (db2 ** 2)



    W1 -= (np.sqrt(D1) + eps) * dW1 * 1 / np.sqrt(v1 + eps)
    W2 -= (np.sqrt(D2) + eps) * dW2 * 1 / np.sqrt(v2 + eps)
    b1 -= (np.sqrt(Db1) + eps) * db1 * 1 / np.sqrt(vb1 + eps)
    b2 -= (np.sqrt(Db2) + eps) * db2 * 1 / np.sqrt(vb2 + eps)
    # D1 = beta * D1 + (1 - beta) * ((w1_old - W1) ** 2)
    # D2 = beta * D2 + (1 - beta) * ((w2_old - W2) ** 2)
    # Db1 = beta * Db1 + (1 - beta) *( (b1_old - b1) ** 2)
    # Db2 = beta * Db2 + (1 - beta) * ((b2_old - b2) ** 2)
    D1 = beta * D1 + (1 - beta) * ((w1_old - W1) ** 2)
    D2 = beta * D2 + (1 - beta) * ((w2_old - W2) ** 2)
    Db1 = beta * Db1 + (1 - beta) * ((b1_old - b1) ** 2)
    Db2 = beta * Db2 + (1 - beta) * ((b2_old - b2) ** 2)


    #
    # W1 = W1 - (np.sqrt(D1)+eps) * dW1 * 1 / np.sqrt(v1 + eps)
    # W2 = W2 - (np.sqrt(D2)+eps) * dW2 * 1 / np.sqrt(v2 + eps)
    # b1 = b1 - (np.sqrt(Db1)+eps) * db1 * 1 / np.sqrt(vb1 + eps)
    # b2 = b2 - (np.sqrt(Db2)+eps) * db2 * 1 / np.sqrt(vb2 + eps)



    # v1=beta*v1+(1-beta)*dW1**2
    # v2=beta*v2+(1-beta)*dW2**2
    # vb1=beta*vb1+(1-beta)*db1**2
    # vb2=beta*vb2+(1-beta)*db2**2



    # W1 = W1 - alpha * v1
    # b1 = b1 - alpha * vb1
    # W2 = W2 - alpha * v2
    # b2 = b2 - alpha * vb2
    # v1 = beta * v1 + (1 - beta) * dW1
    # vb1 = beta * vb1 + (1 - beta) * db1
    # v2 = beta * v2 + (1 - beta) * dW2
    # vb2 = beta * vb2 + (1 - beta) * db2

    return W1, b1, W2, b2, v1, v2, vb1, vb2 ,D1 ,D2 ,Db1 ,Db2


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_parametrs()
    w11 = np.zeros(W1.shape)
    b11 = np.zeros(b1.shape)
    w22 = np.zeros(W2.shape)
    b22 = np.zeros(b2.shape)
    D1 = np.zeros(W1.shape)
    Db1 = np.zeros(b1.shape)
    D2 = np.zeros(W2.shape)
    Db2 = np.zeros(b2.shape)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2, w11, w22, b11, b22,D1,D2,Db1,Db2= update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha, w11, w22, b11,b22,D1,D2,Db1,Db2)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2


W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 100)

Iteration:  0
0.09492857142857143
Iteration:  10
0.09492857142857143
Iteration:  20
0.09495238095238095
Iteration:  30
0.09495238095238095
Iteration:  40
0.09497619047619048
Iteration:  50
0.09497619047619048
Iteration:  60
0.09497619047619048
Iteration:  70
0.095
Iteration:  80
0.095
Iteration:  90
0.095
